In [1]:
import os
from dotenv import load_dotenv

In [2]:
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [3]:
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [6]:
#%cd ..
%pwd

'd:\\GenAIProj\\MyProjs\\Interview-Question-Creator'

In [12]:
from langchain.document_loaders import PyPDFLoader # type: ignore

file_path="data/Java_Cheat_Sheet.pdf"
loader = PyPDFLoader(file_path)
data = loader.load()

In [13]:
len(data)

45

In [18]:
quest_gen = ""
for page in data:
    quest_gen += page.page_content  

In [26]:
from langchain.text_splitter import TokenTextSplitter
""" Keep the chunk size greater than 1. Hence, Chunk size should be smaller."""
que_gen_splitter = TokenTextSplitter(model_name= "gpt-3.5-turbo", chunk_size=1000, chunk_overlap=200)
chunk_ques_gen = que_gen_splitter.split_text(quest_gen)

In [31]:
"""Covert the string to Document for format to be passed to LLM.
Embedding Model doesn't support String."""

#len(chunk_ques_gen) -- list
#type(chunk_ques_gen) -- Str

from langchain.docstore.document import Document
document_ques_gen =  [Document(page_content = ch) for ch in chunk_ques_gen]

In [35]:
""" Initialize the LLM. Lower temperature - More authentic and less creative. Higher temperature - More creative"""

"""The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. 
An updated version of the class exists in the langchain-openai package and should be used instead"""
#from langchain.chat_models import ChatOpenAI - Deprecated Now
from langchain_openai import ChatOpenAI

llm_ques_gen_ppline = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.3)

In [36]:
prompt_template = """
You are an expert at creating interview questions based on the study material and coding material given to you.
Your goal is to prepare a coder or a programmer for their exams and coding tests.
You do this by asking the questions about the text below:

--------
{text}
--------

Create a set of questions that will help the coders and programmers for their tests.
Make sure not to lose any important information.

QUESTIONS:
"""

In [38]:
from langchain.prompts import PromptTemplate
PROMPT_QUESTIONS = PromptTemplate(template=prompt_template, input_variables=['text'])

In [39]:
refine_template = """
You are an expert at creating interview questions based on the study material and coding material given to you.
Your goal is to prepare a coder or a programmer for their exams and coding tests.
We have received some questions meeting the expectations upto a certain extent: {existing_answers}
We have the option to refine the existing questions or add new ones.
(only if necessary) with some more context below

--------
{text}
--------

Given the new context, refine the original questions in English.
If the context is not helpful, please provide the original questions.

QUESTIONS:
"""

In [ ]:
REFINE_PROMPT_QUESTIONS = PromptTemplate(input_variables=["existing_answers", "text"], template=refine_template)